In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

22/08/15 21:35:16 WARN Utils: Your hostname, LAPTOP-3Q5ULU39 resolves to a loopback address: 127.0.1.1; using 172.22.27.113 instead (on interface eth0)
22/08/15 21:35:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/15 21:35:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.json("../../datalake/twitter_nbabrasil")

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- created_at: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- username:

In [7]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{134661711063882...|{[{2021-01-06T00:...|{1559308279301406...|  2022-08-15|
|[{2169254115, 155...|{[{2013-11-05T10:...|{1559260901181628...|  2022-08-15|
|[{57553904, 15589...|{[{2009-07-17T04:...|{1558980506183688...|  2022-08-15|
|[{106824438, 1558...|{[{2010-01-20T20:...|{1558222637159092...|  2022-08-12|
|[{142181522021379...|{[{2021-08-01T12:...|{1558088673694031...|  2022-08-12|
|[{154700803546125...|{[{2022-07-13T00:...|{1557508517631860...|  2022-08-10|
|[{1326384769, 151...|{[{2013-04-04T07:...|{1557363947954642...|  2022-08-10|
|[{3429123275, 155...|{[{2015-08-17T23:...|{1558584012670648...|  2022-08-13|
|[{82350646, 15578...|{[{2009-10-14T12:...|{1557852990899363...|  2022-08-11|
|                null|                null|{null, null, null...|

In [8]:
from pyspark.sql import functions as f

In [9]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [10]:
tweet_df  = df.select(f.explode("data").alias("tweets")).select("tweets.author_id", "tweets.conversation_id",
                                                                "tweets.created_at", "tweets.id",
                                                                "tweets.public_metrics.*", "tweets.text")

In [11]:
tweet_df.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [12]:
df.select(f.explode("includes.users")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [13]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [14]:
user_df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [16]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')

In [17]:
user_df.coalesce(1).write.mode("overwrite").json('output/user')